In [1]:
import os
download = not os.path.isfile('data/nmnist_test.zip')
download

import tonic
import tonic.transforms as transforms

testset = tonic.datasets.NMNIST(save_to='./data', download=download, train=False)
testloader = tonic.datasets.DataLoader(testset,
                                       batch_size=1,
                                       shuffle=True)

events, target = next(iter(testloader))

In [2]:
%run boilerplate.ipynb

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 7.18.1

numpy     : 1.19.2
matplotlib: 3.3.2
pyNN      : 0.9.5
neo       : 0.8.0
moviepy   : 1.0.3
imageio   : 2.9.0

Compiler    : GCC 9.3.0
OS          : Linux
Release     : 5.4.0-53-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 4
Architecture: 64bit

Hostname: inv-ope-de06

Git hash: 4b4c65e8f06f3149c6e7814c12e967d55b9e0e2e

Git repo: https://github.com/SpikeAI/2020-11_brainhack_Project7

Git branch: main



In [3]:
testset.sensor_size, testset.ordering

((34, 34), 'xytp')

In [4]:
N_pop = np.prod(testset.sensor_size)
N_pop

1156

In [5]:
events.shape

torch.Size([1, 2901, 4])

Timings are in microseconds

In [6]:
events[0, :, testset.ordering.find("t")]*1e-6

tensor([0.0044, 0.0052, 0.0058,  ..., 0.3027, 0.3053, 0.3059])

In [7]:
index_x = testset.ordering.find("x")
index_y = testset.ordering.find("y")
index_t = testset.ordering.find("t")
index_x, index_y, index_t

(0, 1, 2)

In [8]:
events[0, :, index_x], events[0, :, index_y] 

(tensor([19, 18, 19,  ..., 22,  8, 33], dtype=torch.int32),
 tensor([16, 30, 20,  ..., 11, 11, 25], dtype=torch.int32))

In [9]:
events[0, :, index_x] == events[0, 0, index_x]

tensor([ True, False,  True,  ..., False, False, False])

In [10]:
events[0, :, index_y] == events[0, :, index_y]

tensor([True, True, True,  ..., True, True, True])

In [11]:
(events[0, :, index_x] == events[0, :, index_x]) & (events[0, :, index_y] == events[0, :, index_y])

tensor([True, True, True,  ..., True, True, True])

In [12]:
np.where((events[0, :, index_x] == events[0, :, index_x]) & (events[0, :, index_y] == events[0, :, index_y]))[0]

array([   0,    1,    2, ..., 2898, 2899, 2900])

In [13]:
events[0, np.where((events[0, :, index_x] == events[0, :, index_x]) & (events[0, :, index_y] == events[0, :, index_y]))[0], index_t ]

tensor([  4430,   5237,   5779,  ..., 302699, 305312, 305912],
       dtype=torch.int32)

In [14]:
events_np = events.numpy()
cellSourceSpikes = []
for i in range(N_pop):
    row = i//(testset.sensor_size[0])
    col = i%(testset.sensor_size[0])
    spike_idx = np.where((events_np[0, :, index_x] == row) & (events_np[0, :, index_y] == col))[0]
    spike_times = events_np[0, spike_idx, index_t] * 1.e-3 # in milliseconds
    cellSourceSpikes.append(spike_times)

cellSourceSpikes = [list(elem) for elem in cellSourceSpikes]
idx = events[0, 0, index_x]*(testset.sensor_size[0]) + events[0, 0, index_y] # first spike
len(cellSourceSpikes), cellSourceSpikes[idx], np.sum([len(st) for st in cellSourceSpikes])

(1156,
 [4.43,
  46.005,
  49.987,
  67.01,
  73.122,
  152.729,
  248.547,
  253.359,
  256.815,
  260.26800000000003,
  281.295,
  295.78700000000003],
 2901)

In [15]:
try:
    import pyNN.spiNNaker as sim
    simulator = 'spinnaker'
except ModuleNotFoundError:
    import pyNN.nest as sim
    simulator = 'nest'

CSAConnector: libneurosim support not available in NEST.
Falling back on PyNN's default CSAConnector.
Please re-compile NEST using --with-libneurosim=PATH


In [16]:
sim.setup(timestep=dt,
          min_delay=1,
          max_delay=100,
          debug=0)

sources = sim.SpikeSourceArray(spike_times=cellSourceSpikes)
spikeSource = sim.Population(N_pop, sources)    
spikeSource.record(['spikes'])

In [17]:
sim.run(simtime=simtime)

350.0

In [18]:
spikeSources  = spikeSource.get_data()#.segments[0].spiketrains
S_spikes = spikeSources.segments[0].spiketrains

In [19]:
sim.end()

In [20]:
st = S_spikes[idx]
st.as_array()

array([  4.43 ,  46.005,  49.987,  67.01 ,  73.122, 152.729, 248.547,
       253.359, 256.815, 260.268, 281.295, 295.787])

In [21]:
def spikelist2numpy(spikelist):
    output_spike = np.zeros((testset.sensor_size[0], testset.sensor_size[1], time_bins))
    for i, spiketrain in enumerate(spikelist):
        row = i//(testset.sensor_size[0])
        col = i%(testset.sensor_size[0])
        for spiketime in spiketrain.as_array():
            # print(int(spiketime))
            output_spike[row, col, int(spiketime)] = 1
    return output_spike

In [22]:
nmnist_spike = spikelist2numpy(S_spikes)
minmax(nmnist_spike)

movie.min()=0.0 - movie.max()=1.0 


In [23]:
show(make_movie(nmnist_spike, label='nmnist_spike'))

t:  22%|██▏       | 77/350 [00:00<00:00, 763.10it/s, now=None]

Moviepy - Building video output/nmnist_spike.mp4.
Moviepy - Writing video output/nmnist_spike.mp4



Moviepy - Done !
Moviepy - video ready output/nmnist_spike.mp4


In [24]:
nmnist_spike.shape

(34, 34, 350)

In [25]:
nmnist_spike.sum()

2897.0

# NCARS

In [ ]:
download = not os.path.isfile('data/Prophesee_Dataset_n_cars.zip')

testset = tonic.datasets.NCARS(save_to='./data', download=download, train=False)
testloader = tonic.datasets.DataLoader(testset,
                                       batch_size=1,
                                       shuffle=True)

events, target = next(iter(testloader))

In [ ]:
events_np = events.numpy()
cellSourceSpikes = []
simtime = 0
for i in range(N_pop):
    row = i//(testset.sensor_size[0])
    col = i%(testset.sensor_size[0])
    spike_idx = np.where((events_np[0, :, index_x] == row) & (events_np[0, :, index_y] == col))[0]
    spike_times = events_np[0, spike_idx, index_t] * 1.e-3 # in milliseconds
    cellSourceSpikes.append(spike_times)
    simtime = np.max(simtime, np.max(spike_times))

cellSourceSpikes = [list(elem) for elem in cellSourceSpikes]
idx = events[0, 0, index_x]*(testset.sensor_size[0]) + events[0, 0, index_y] # first spike
len(cellSourceSpikes), cellSourceSpikes[idx], np.sum([len(st) for st in cellSourceSpikes])

In [ ]:
sim.setup(timestep=dt,
          min_delay=1,
          max_delay=100,
          debug=0)

sources = sim.SpikeSourceArray(spike_times=cellSourceSpikes)
spikeSource = sim.Population(N_pop, sources)    
spikeSource.record(['spikes'])

In [17]:
sim.run(simtime=simtime)

350.0

In [18]:
spikeSources  = spikeSource.get_data()#.segments[0].spiketrains
S_spikes = spikeSources.segments[0].spiketrains

In [19]:
sim.end()

In [20]:
st = S_spikes[idx]
st.as_array()

array([  4.43 ,  46.005,  49.987,  67.01 ,  73.122, 152.729, 248.547,
       253.359, 256.815, 260.268, 281.295, 295.787])

In [21]:
def spikelist2numpy(spikelist):
    output_spike = np.zeros((testset.sensor_size[0], testset.sensor_size[1], time_bins))
    for i, spiketrain in enumerate(spikelist):
        row = i//(testset.sensor_size[0])
        col = i%(testset.sensor_size[0])
        for spiketime in spiketrain.as_array():
            # print(int(spiketime))
            output_spike[row, col, int(spiketime)] = 1
    return output_spike

In [22]:
nmnist_spike = spikelist2numpy(S_spikes)
minmax(nmnist_spike)

movie.min()=0.0 - movie.max()=1.0 
